In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv('../few_shot_dev_results.csv')

In [3]:
df.head()

,model,prompt_type,prompt_index,split,run,TP,FP,FN,Prec,Rec,F0.5,GLEU,prompt_template,prompt_type_index
0,facebook/opt-iml-max-30b,3-shot,1,jfleg-dev,1,839,192,992,0.8138,0.4582,0.7045,0.498200,Reply with a corrected version of the input se...,3-shot_1
1,facebook/opt-iml-max-30b,2-shot-Coyne,1,jfleg-dev,1,919,264,957,0.7768,0.4899,0.6954,0.506410,Reply with a corrected version of the input se...,2-shot-Coyne_1
2,facebook/opt-iml-max-30b,3-shot,3,jfleg-dev,1,821,211,1005,0.7955,0.4496,0.6895,0.494307,You are an English language teacher. A student...,3-shot_3
3,facebook/opt-iml-max-30b,1-shot,3,jfleg-dev,1,710,176,1018,0.8014,0.4109,0.6734,0.470543,You are an English language teacher. A student...,1-shot_3
4,facebook/opt-iml-max-30b,1-shot,1,jfleg-dev,1,637,128,1038,0.8327,0.3803,0.6727,0.461363,Reply with a corrected version of the input se...,1-shot_1


In [4]:
# function to return a table of max F0.5 scores for each model, filtered by split
def get_max_f05(df, split):
    # filter by split
    df_split = df[df['split'] == split]
    # group df by model and select the rows with the maximum F0.5 score
    max_metric = 'F0.5'
    if 'jfleg' in split:
        max_metric = 'GLEU'

    idx = df_split.groupby(['model'])[max_metric].transform(max) == df_split[max_metric]
    df_max = df_split[idx]
    return df_max

In [5]:
splits = df.split.unique()
splits

array(['jfleg-dev', 'wibea-dev', 'fce-dev'], dtype=object)

In [8]:
df_max_splits = {}
for split in splits:
    # metric is gleu if split is jfleg
    metric = 'F0.5' if 'jfleg' not in split else 'GLEU'
    df_temp = get_max_f05(df, split)
    df_max_splits[split] = df_temp[['model', 'prompt_type_index', metric, 'Prec', 'Rec']]
    # convert metric to float
    df_max_splits[split][metric] = df_max_splits[split][metric].astype(float)
    # format metric to 3 decimal places
    df_max_splits[split][metric] = df_max_splits[split][metric].apply(lambda x: round(x, 3))


/tmp/ipykernel_2829003/856668224.py:10: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  idx = df_split.groupby(['model'])[max_metric].transform(max) == df_split[max_metric]
/tmp/ipykernel_2829003/2860735688.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_max_splits[split][metric] = df_max_splits[split][metric].astype(float)
/tmp/ipykernel_2829003/2860735688.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [9]:
df_max_splits['jfleg-dev']

,model,prompt_type_index,GLEU,Prec,Rec
1,facebook/opt-iml-max-30b,2-shot-Coyne_1,0.506,0.7768,0.4899
15,gpt-3.5-turbo-0613,4-shot_2,0.577,0.6356,0.6377
21,google/flan-t5-xxl,1-shot_2,0.463,0.7792,0.3593
26,stabilityai/StableBeluga2,4-shot_1,0.560,0.6312,0.6174
29,tiiuae/falcon-40b-instruct,4-shot_1,0.548,0.6474,0.5521
55,command,2-shot-Coyne_1,0.543,0.5948,0.5873
65,Writer/InstructPalmyra-20b,2-shot-Coyne_1,0.513,0.5630,0.5554
72,meta-llama/Llama-2-70b-chat-hf,2-shot_2,0.469,0.5408,0.6086
73,bigscience/bloomz-7b1,2-shot-Coyne_1,0.456,0.6389,0.3588


In [10]:
all_models = df.model.unique()
all_models

array(['facebook/opt-iml-max-30b', 'tiiuae/falcon-40b-instruct',
       'gpt-3.5-turbo-0613', 'google/flan-t5-xxl',
       'stabilityai/StableBeluga2', 'Writer/InstructPalmyra-20b',
       'command', 'bigscience/bloomz-7b1',
       'meta-llama/Llama-2-70b-chat-hf'], dtype=object)

In [11]:
# filter df by model and split
def filter_df(df, model, split):
    df_model = df[df['model'] == model]
    df_model_split = df_model[df_model['split'] == split]
    if 'jfleg' in split:
        # sort dataframe by GLEU
        df_model_split = df_model_split.sort_values(by=['GLEU'], ascending=False)
    return df_model_split


In [12]:
# write a latex row using the format: \diagbox[dir=SW]{metric}{prompt_index}
# where each row is a model and each column is a split
def write_latex_row_subcript(df, model, split):
    metric = 'F0.5' if 'jfleg' not in split else 'GLEU'
    # get the metric value for the model from the df
    metric_value = df[df['model'] == model][metric].values[0]
    
    # check if metric_value is the max value for this dataframe
    max_metric_value = df[metric].max()
    if metric_value == max_metric_value:
        metric_value = '\\textbf{' + '{:.3f}'.format(metric_value) + '}'
    else:
        metric_value = '{:.3f}'.format(metric_value)

    # get the prompt_index for the model from the df
    prompt_index = df[df['model'] == model]['prompt_type_index'].values[0]

    if 'Coyne' in prompt_index:
        prompt_index = '2-shot_C'

    # write the latex row
    return f'{str(metric_value)}$_{{{str(prompt_index)}}}$'

In [13]:
# write a latex row using the format: \diagbox[dir=SW]{metric}{prompt_index}
# where each row is a model and each column is a split
def write_latex_row_expanded_subcript(df, model, split):
    metric = 'F0.5' if 'jfleg' not in split else 'GLEU'
    # get the metric value for the model from the df
    metric_value = df[df['model'] == model][metric].values[0]
    
    # check if metric_value is the max value for this dataframe
    max_metric_value = df[metric].max()
    if metric_value == max_metric_value:
        metric_value = '\\textbf{' + '{:.3f}'.format(metric_value) + '}'
    else:
        metric_value = '{:.3f}'.format(metric_value)

    # get the prompt_index for the model from the df
    prompt_index = df[df['model'] == model]['prompt_type_index'].values[0]


    prompt_type = prompt_index.split('_')[0] 
    prompt_index = prompt_index.split('_')[1]

    if 'Coyne' in prompt_type:
        prompt_type = '2'
        prompt_index = '\\textsc{coyne}$^{*}$'
    else:
        prompt_type = prompt_type.split('-')[0]
        if int(prompt_index) == 1:
            prompt_index = '\\textsc{coyne}'
        elif int(prompt_index) == 2:
            prompt_index = '\\textsc{tool}'
        else:
            prompt_index = '\\textsc{elt}'
        
    # write the latex row
    # $_{{{str(prompt_index)}}}$
    return f'{str(metric_value)} & {prompt_type} & {str(prompt_index)}'

In [14]:

# write the latex table
# print(' & '.join([''] + [split for split in sorted(splits)]) + ' \\\\')

model_order = [
    'bigscience/bloomz-7b1',
    'google/flan-t5-xxl',
    'Writer/InstructPalmyra-20b',
    'facebook/opt-iml-max-30b',
    'tiiuae/falcon-40b-instruct',
    'meta-llama/Llama-2-70b-chat-hf',
    'stabilityai/StableBeluga2',
    'command', 
    'gpt-3.5-turbo-0613',
    # 'gpt-4-0613',
    ]

splits = df.split.unique()

for model in model_order:
    if model not in all_models:
        continue
    model_name = model.split('/')[-1]
    row = [model_name]
    for split in sorted(splits):
        row.append(write_latex_row_expanded_subcript(df_max_splits[split], model, split))
    
    print(' & '.join(row) + ' \\\\')
    # print('\\hline')

bloomz-7b1 & 0.349 & 3 & \textsc{coyne} & 0.456 & 2 & \textsc{coyne}$^{*}$ & 0.347 & 3 & \textsc{coyne} \\
flan-t5-xxl & \textbf{0.447} & 1 & \textsc{tool} & 0.463 & 1 & \textsc{tool} & 0.423 & 3 & \textsc{tool} \\
InstructPalmyra-20b & 0.341 & 2 & \textsc{coyne} & 0.513 & 2 & \textsc{coyne}$^{*}$ & 0.374 & 2 & \textsc{coyne} \\
opt-iml-max-30b & 0.382 & 2 & \textsc{coyne}$^{*}$ & 0.506 & 2 & \textsc{coyne}$^{*}$ & 0.400 & 3 & \textsc{elt} \\
falcon-40b-instruct & 0.425 & 2 & \textsc{tool} & 0.548 & 4 & \textsc{coyne} & \textbf{0.454} & 4 & \textsc{tool} \\
Llama-2-70b-chat-hf & 0.299 & 1 & \textsc{coyne} & 0.469 & 2 & \textsc{tool} & 0.330 & 2 & \textsc{coyne} \\
StableBeluga2 & 0.375 & 4 & \textsc{coyne} & 0.560 & 4 & \textsc{coyne} & 0.440 & 4 & \textsc{coyne} \\
command & 0.332 & 2 & \textsc{coyne}$^{*}$ & 0.543 & 2 & \textsc{coyne}$^{*}$ & 0.380 & 2 & \textsc{coyne}$^{*}$ \\
gpt-3.5-turbo-0613 & 0.408 & 1 & \textsc{elt} & \textbf{0.577} & 4 & \textsc{tool} & 0.439 & 1 & \textsc{to

In [15]:
df_max_splits[split]

,model,prompt_type_index,F0.5,Prec,Rec
110,tiiuae/falcon-40b-instruct,4-shot_2,0.454,0.4669,0.4075
116,stabilityai/StableBeluga2,4-shot_1,0.440,0.4276,0.4964
118,gpt-3.5-turbo-0613,1-shot_2,0.439,0.4222,0.5236
138,google/flan-t5-xxl,3-shot_2,0.423,0.6227,0.1854
160,facebook/opt-iml-max-30b,3-shot_3,0.400,0.5770,0.1796
182,command,2-shot-Coyne_1,0.380,0.3684,0.4332
190,Writer/InstructPalmyra-20b,2-shot_1,0.374,0.3958,0.3050
222,bigscience/bloomz-7b1,3-shot_1,0.347,0.5084,0.1529
240,meta-llama/Llama-2-70b-chat-hf,2-shot_1,0.330,0.3063,0.4794


In [16]:
# for each dataframe in df_max_splits, add a column with the split name
for split in sorted(splits):
    df_max_splits[split]['split'] = split

# concatenate all dataframes from df_max_splits
df_max_all = pd.concat(df_max_splits.values(), axis=0)

/tmp/ipykernel_2829003/2242612188.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_max_splits[split]['split'] = split


In [17]:
df_max_all.to_csv('../best_few_shot_dev_results.csv', index=False)